<a href="https://colab.research.google.com/github/Doni0007/DESAFIOS_DIO/blob/main/desafio_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Visão geral do ETL**

🔹 *Fase E — Extract*

Ler dois arquivos CSV:

id_nomes_clientes → id_cliente, nome

dados_contas.csv → id_cliente, idade, profissão, estado civil, formação, etc.

🔹 Fase T — Transform

Join dos arquivos pelo id_cliente

Filtrar clientes com:

saldo_conta_corrente > 5000

Criar mensagem personalizada usando:

Nome

Perfil do cliente

Sugestão de investimento

Uso da API da OpenAI para gerar textos naturais e personalizados

🔹 Fase L — Load

Salvar resultado em:

clientes_investimento.c

1️⃣ ** Montar o Google Drive **

  *Se necessário, autorize o acesso*

In [29]:
#
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


2️⃣ **Estrutura recomendada no Drive**

*Meu Drive/*
 projeto_etl_banco/
     *input/*
     *clientes.csv*
     *│dados_clientes.csv*
     *output/*


3️⃣ **Código ETL**

📦 Imports e paths

In [46]:
import pandas as pd
import os
import openai
from openai import OpenAI
from dotenv import load_dotenv

4️⃣**Fase E — Extract**

In [42]:
import pandas as pd
import os

BASE_PATH = "/content/drive/MyDrive/etl_banco"
INPUT_PATH = os.path.join(BASE_PATH, "input")

clientes = pd.read_csv(os.path.join(INPUT_PATH, "id_nomes_clientes.csv"), encoding='latin1', sep=';')
dados = pd.read_csv(os.path.join(INPUT_PATH, "dados_contas.csv"), encoding='latin1', sep=';')

print("Arquivos carregados com sucesso!")

Arquivos carregados com sucesso!


5️⃣ **Fase T — Transform**

In [43]:
df = clientes.merge(dados, on="idCliente", how="inner")

df_investidores = df[df["Saldo_Conta_Corrente"].str.replace('R$ ', '').str.replace('.', '').str.replace(',', '.').astype(float) > 5000].copy()

print(f"Clientes elegíveis para investimento: {len(df_investidores)}")

Clientes elegíveis para investimento: 4


6️⃣ **Configurar OpenAI no Colab**

In [55]:
from getpass import getpass

openai.api_key = getpass("Digite sua OpenAI API Key: ")


Digite sua OpenAI API Key: ··········


7️⃣ **Gerar mensagens personalizadas (OpenAI)**

In [56]:
def gerar_mensagem(cliente):
    prompt = f"""
    Gere uma mensagem curta, profissional e consultiva para um cliente bancário.

    Nome: {cliente['nome completo']}
    Idade: {cliente['Idade']}
    Profissão: {cliente['Profissão']}
    Formação: {cliente['Formação']}
    Saldo em conta corrente: {cliente['Saldo_Conta_Corrente']}

    Objetivo: sugerir investimentos utilizando as carteiras do banco,
    com linguagem clara, personalizada e não agressiva.
    """

    client = OpenAI(api_key=openai.api_key) # Pass the API key explicitly
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Você é um consultor financeiro de um grande banco."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.6
    )

    return response.choices[0].message.content.strip()

*Aplicar a função*

In [57]:
df_investidores["mensagem_investimento"] = df_investidores.apply(
    gerar_mensagem,
    axis=1
)

8️⃣ **Fase L — Load**

In [58]:
OUTPUT_PATH = os.path.join(BASE_PATH, "output") # Ensure OUTPUT_PATH is defined if not already
output_file = os.path.join(OUTPUT_PATH, "clientes_investimento.csv")

df_investidores.to_csv(
    output_file,
    index=False,
    encoding="utf-8"
)

print("Arquivo gerado com sucesso!")
print(output_file)

Arquivo gerado com sucesso!
/content/drive/MyDrive/etl_banco/output/clientes_investimento.csv
